In [0]:
import psycopg2
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
def call_df(table_name):
    with open('config.json', 'r') as f:
        config = json.load(f)
        
    conn = psycopg2.connect(user = config['USER'],
                              password = config['PASSWORD'],
                              host = config['HOST'],
                              port = config['PORT'],
                              database = config['DATABASE'])
    
    sql = f'SELECT * FROM {table_name}'
    df = pd.read_sql_query(sql, conn)
    conn.close()
    return df

In [0]:
district = call_df('crawling_db.district_table')
apartment = call_df('crawling_db.apartment_table').drop(columns='table_id')
school = call_df('crawling_db.school_table').drop(columns='table_id')
price = call_df('crawling_db.price_table')
subway = call_df('crawling_db.subway_table').drop(columns='table_id')

In [0]:
df = (price.merge(apartment, how='left', on='apartment_id').
      merge(district, how='left', on='district_id').
      merge(school, how='left', on='apartment_id').
      merge(subway, how='left', on='apartment_id'))

In [0]:
import re
df['area'] = df['area'].apply(lambda x: int(re.split('\D',x)[0]))

In [0]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

label_encoder.fit(df.apartment_addr_town)
apartment_town_label = label_encoder.transform(df.apartment_addr_town)
townlabel_df = pd.DataFrame([apartment_town_label,df['apartment_addr_town']])

In [0]:
label_encoder.fit(df.apartment_name)
apartment_name_label = label_encoder.transform(df.apartment_name)
apartment_label_df = pd.DataFrame([apartment_name_label,df['apartment_name']])

In [0]:
townlabel_df = np.transpose(townlabel_df)
apartment_label_df = np.transpose(apartment_label_df)

In [58]:
#행정동 별 모델을 만들었기 때문에, label dataframe 미리 생성
townlabel_df = townlabel_df.drop_duplicates()
townlabel_df.columns = ['label','town_name']
townlabel_df.head()

,label,town_name
0,8,개포동
3775,44,논현동
5636,58,대치동
9423,61,도곡동
13157,133,삼성동


In [59]:
apartment_label_df = apartment_label_df.drop_duplicates()
apartment_label_df.columns = ['label','apartment_name']
apartment_label_df.head()

,label,apartment_name
0,41,LG개포자이
99,229,개포래미안포레스트
112,231,개포상지리츠빌
119,235,개포주공1단지
721,236,개포주공4단지


In [0]:
df['school_students']=pd.to_numeric(df['school_students'])

#범주형 자료 label하기 

df.apartment_addr_town = apartment_town_label
df.apartment_name = apartment_name_label

label_encoder.fit(df.apartment_builder)
df.apartment_builder = label_encoder.transform(df.apartment_builder)

label_encoder.fit(df.st_name)
df.st_name = label_encoder.transform(df.st_name)


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 457904 entries, 0 to 457903
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   price_id               457904 non-null  int64  
 1   apartment_id           457904 non-null  int64  
 2   area                   457904 non-null  int64  
 3   period                 457904 non-null  object 
 4   year                   457904 non-null  int64  
 5   month                  457904 non-null  int64  
 6   amount                 457904 non-null  int64  
 7   amount_original        457904 non-null  object 
 8   district_id            457904 non-null  int64  
 9   apartment_addr_town    457904 non-null  int64  
 10  apartment_builder      457904 non-null  int64  
 11  apartment_build_year   457904 non-null  int64  
 12  apartment_build_month  457904 non-null  int64  
 13  apartment_floor_min    457904 non-null  object 
 14  apartment_floor_max    457904 non-nu

In [63]:
df = df.drop(['period','district_name','apartment_id','price_id','amount_original'],axis=1)
df = df.drop(['apartment_floor_min','apartment_floor_max'],axis=1)
df = df.drop(['school_name','school_dist','school_addr_district','school_addr_town'],axis=1)
df.head()

,area,year,month,amount,district_id,apartment_addr_town,apartment_builder,apartment_build_year,apartment_build_month,apartment_parking,apartment_name,school_students,st_name,st_dist,st_volume
0,160,2020,2,2060000000,1,8,779,2004,6,2.36,41,316.0,277,0.002592,1
1,160,2020,1,2030000000,1,8,779,2004,6,2.36,41,316.0,277,0.002592,1
2,160,2019,11,1950000000,1,8,779,2004,6,2.36,41,316.0,277,0.002592,1
3,160,2019,10,1939999999,1,8,779,2004,6,2.36,41,316.0,277,0.002592,1
4,160,2019,7,1739999999,1,8,779,2004,6,2.36,41,316.0,277,0.002592,1


In [66]:
#파생변수 생성
# 아파트 연도별 거래 총액, 거래수
district = df.groupby(['apartment_name','year']).sum()['amount'] #변동성 계산을 위해 우선 sum의 값으로 도출
count = df.groupby(['apartment_name','year']).count()['amount']
df2 = pd.DataFrame(district)
df2['trade_n'] = count
df2.tail()

amount  trade_n
apartment_name year                     
4857           2017   710000000        1
4858           2009  4459999997        5
               2010  1740000000        2
               2012   800000000        1
               2018   869999999        1

In [65]:
#아파트 별 거래액 평균, 거래수
district = df.groupby(['apartment_name']).mean()['amount'] #knn에 사용하기 위해 mean값 사용
count = df.groupby(['apartment_name']).count()['amount']
df3 = pd.DataFrame(district)
df3['trade_n'] = count #거래량
df3.head()

,amount,trade_n
apartment_name,,
0,1.710227e+08,22
1,2.990500e+08,4
2,2.861333e+08,6
3,4.027145e+08,78
4,9.908333e+08,12


In [75]:
# 파생변수1 : amount 변동비율 
# 변동비율 계산 시 2020년 자료는 제외
ini = 0
df3['amount_volatility']=0
for i in range(4859):
      df_apt = df2.loc[i]
      num = df2.loc[i]['amount'].count()
      if num == 1 : df3['amount_volatility'].iloc[i]=1
      else:
        for j in range(num-1):
            dif = df_apt.iloc[j+1]['amount']-df_apt.iloc[j]['amount']
            ini = abs(dif) + ini
        vol = ini/(num-1)
        mean = df_apt['amount'].mean()
        df3['amount_volatility'].iloc[i]=mean/vol

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [91]:
# 파생변수 2: 거래수 변동비율 계산 
# 변동비율 계산 시 2020년 자료는 제외
ini = 0
df3['trade_n_volatility']=0
for i in range(4859):
      df_apt = df2.loc[i]
      num = df2.loc[i]['trade_n'].count()
      if num == 1 : df3['trade_n_volatility'].iloc[i]=1
      else:
        for j in range(num-1):
            dif = df_apt.iloc[j+1]['trade_n']-df_apt.iloc[j]['trade_n']
            ini = abs(dif) + ini
        vol = ini/(num-1)
        mean = df_apt['trade_n'].mean()
        df3['trade_n_volatility'].iloc[i]=mean/vol

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [79]:
df3.head()

,amount,trade_n,amount_volatility,trade_n_volatility
apartment_name,,,,
0,1.710227e+08,22,1.771630,2.200000
1,2.990500e+08,4,0.345763,0.266667
2,2.861333e+08,6,0.250321,0.214286
3,4.027145e+08,78,1.276946,1.090909
4,9.908333e+08,12,0.388043,0.154286


In [138]:
#거시경제 변수 추가
economy = pd.read_csv("economic_indicators.csv")
economy.head()

,year,Nominal_GDP,RealGDP_growth_rate,key_interest_rate,price_index
1,2006,1005602,5.3,4.5,0.4
2,2007,1089660,5.8,5.0,1.8
3,2008,1154217,3.0,3.0,2.3
4,2009,1205348,0.8,2.0,1.6
5,2010,1322611,6.8,2.5,1.9


In [0]:
#경제지표, 변동비율 데이터프레임 merge
new_df = pd.merge(df,economy,on='year',how='left')
new_df = pd.merge(new_df,df3[['amount_volatility','trade_n_volatility']],on='apartment_name',how='left')
#2020년 경제지표는 불확실해서 우선 2020년을 제외하고 모델링
new_df = new_df[new_df['year']<2020]
new_df = new_df.dropna()

In [221]:
new_df.head()

,area,year,month,amount,district_id,apartment_addr_town,apartment_builder,apartment_build_year,apartment_build_month,apartment_parking,apartment_name,school_students,st_name,st_dist,st_volume,Nominal_GDP,RealGDP_growth_rate,key_interest_rate,price_index,amount_volatility,trade_n_volatility
2,160,2019,11,1950000000,1,8,779,2004,6,2.36,41,316.0,277,0.002592,1,1913964,2.0,1.25,-0.1,0.240517,0.078107
3,160,2019,10,1939999999,1,8,779,2004,6,2.36,41,316.0,277,0.002592,1,1913964,2.0,1.25,-0.1,0.240517,0.078107
4,160,2019,7,1739999999,1,8,779,2004,6,2.36,41,316.0,277,0.002592,1,1913964,2.0,1.25,-0.1,0.240517,0.078107
5,160,2018,8,1789999999,1,8,779,2004,6,2.36,41,316.0,277,0.002592,1,1893497,2.7,1.75,0.6,0.240517,0.078107
6,160,2018,6,1620000000,1,8,779,2004,6,2.36,41,316.0,277,0.002592,1,1893497,2.7,1.75,0.6,0.240517,0.078107


In [222]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447831 entries, 2 to 457903
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   area                   447831 non-null  int64  
 1   year                   447831 non-null  int64  
 2   month                  447831 non-null  int64  
 3   amount                 447831 non-null  int64  
 4   district_id            447831 non-null  int64  
 5   apartment_addr_town    447831 non-null  int64  
 6   apartment_builder      447831 non-null  int64  
 7   apartment_build_year   447831 non-null  int64  
 8   apartment_build_month  447831 non-null  int64  
 9   apartment_parking      447831 non-null  float64
 10  apartment_name         447831 non-null  int64  
 11  school_students        447831 non-null  float64
 12  st_name                447831 non-null  int64  
 13  st_dist                447831 non-null  float64
 14  st_volume              447831 non-nu

# Gradient Boosting Regressor

In [190]:
#행정동 별 모델링
df_apartment = new_df[new_df['apartment_addr_town']==1]
df_apartment.head()
df_apartment = df_apartment.drop(['district_id','apartment_addr_town'],axis=1)

,area,year,month,amount,apartment_builder,apartment_build_year,apartment_build_month,apartment_parking,apartment_name,school_students,st_name,st_dist,st_volume,Nominal_GDP,RealGDP_growth_rate,key_interest_rate,price_index,amount_volatility,trade_n_volatility
103397,84,2019,8,280000000,2506,1998,2,1.0,4781,333.0,49,0.008934,1,1913964,2.0,1.25,-0.1,0.000199,0.000446
103398,84,2018,6,264000000,2506,1998,2,1.0,4781,333.0,49,0.008934,1,1893497,2.7,1.75,0.6,0.000199,0.000446
103399,84,2018,4,295000000,2506,1998,2,1.0,4781,333.0,49,0.008934,1,1893497,2.7,1.75,0.6,0.000199,0.000446
103400,84,2018,3,295000000,2506,1998,2,1.0,4781,333.0,49,0.008934,1,1893497,2.7,1.75,0.6,0.000199,0.000446
103401,84,2018,2,270000000,2506,1998,2,1.0,4781,333.0,49,0.008934,1,1893497,2.7,1.75,0.6,0.000199,0.000446


In [0]:
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#scaler.fit(df_apartment)

In [0]:
from sklearn.model_selection import train_test_split

x = df_apartment.drop('amount',axis=1)
y = df_apartment['amount']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

In [232]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2,n_estimators=3,learning_rate=0.1,random_state=42)
gbrt.fit(x_train,y_train)
y_predict = gbrt.predict(x_test)
pd.DataFrame([y_predict,y_test])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,2.261363e+08,2.347107e+08,2.285945e+08,2.347107e+08,2.285945e+08,2.261363e+08,2.209761e+08,2.347107e+08,2.347107e+08,2.347107e+08,2.444121e+08,2.285945e+08,2.285945e+08,2.142126e+08,2.444121e+08,2.285945e+08,2.347107e+08,2.559784e+08,2.347107e+08,2.444121e+08,2.261363e+08,2.347107e+08
1,1.700000e+08,2.950000e+08,2.000000e+08,2.710000e+08,2.000000e+08,2.200000e+08,2.000000e+08,2.550000e+08,2.500000e+08,2.650000e+08,2.640000e+08,2.000000e+08,1.890000e+08,1.445000e+08,2.950000e+08,1.920000e+08,2.570000e+08,3.400000e+08,2.900000e+08,2.950000e+08,2.180000e+08,2.660000e+08


# Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [197]:
forest = RandomForestClassifier(n_estimators=100)
forest.fit(x_train,y_train)
y_predict = forest.predict(x_test)
pd.DataFrame([y_predict,y_test])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,195000000,256999999,199000000,270000000,215000000,190000000,183000000,270000000,270000000,270000000,270000000,199000000,256000000,130000000,270000000,210300000,215000000,341000000,275000000,270000000,190000000,280000000
1,170000000,295000000,200000000,271000000,200000000,220000000,200000000,254999999,250000000,265000000,264000000,200000000,189000000,144500000,295000000,192000000,256999999,340000000,290000000,295000000,218000000,266000000


# Example

2020년도를 예측하기 위해서는 거시경제지수에 대한 예측값이 필요(추후에 경제학자들이 예측한 수치를 사용하면 될듯..)

편의상 2019년 6월 가리봉동 구로두산위브 53면적 가격과 실제값을 비교해본다고 하자

In [154]:
apartment_label_df.head()

,label,apartment_name
0,41,LG개포자이
99,229,개포래미안포레스트
112,231,개포상지리츠빌
119,235,개포주공1단지
721,236,개포주공4단지


In [152]:
townlabel_df.head()

,label,town_name
0,8,개포동
3775,44,논현동
5636,58,대치동
9423,61,도곡동
13157,133,삼성동


In [201]:
townlabel_df[townlabel_df['town_name']=='가리봉동']

,label,town_name
103396,1,가리봉동


In [199]:
apartment_label_df[apartment_label_df['apartment_name']=='구로두산위브']

,label,apartment_name
113425,448,구로두산위브


In [225]:
# 2019년 면적53 6월
ex = new_df[(new_df['apartment_name']==448)&(new_df['area']==53)&(new_df['year']==2019)&(new_df['month']==6)]
ex = ex.drop(['district_id','apartment_addr_town'],axis=1)
ex

,area,year,month,amount,apartment_builder,apartment_build_year,apartment_build_month,apartment_parking,apartment_name,school_students,st_name,st_dist,st_volume,Nominal_GDP,RealGDP_growth_rate,key_interest_rate,price_index,amount_volatility,trade_n_volatility
113434,53,2019,6,332000000,1215,2006,7,0.87,448,567.0,49,0.001102,1,1913964,2.0,1.25,-0.1,0.010001,0.017542


In [229]:
x = ex.drop('amount',axis=1)
y_real = ex['amount']
y_predict = forest.predict(x)
pd.DataFrame([y_predict,y_real])

,0
0,280000000
1,332000000
